https://colab.research.google.com/drive/13ErkLg5FZHIbnUGZRkKlL-9WNCNQPIow#scrollTo=H5t4_7HdjYEE


1.   pip install transformers 






In [7]:
import pandas as pd
import matplotlib
import numpy as np
import torch
import transformers
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

In [8]:
batch_size = 6
max_sequence_length = 512
learning_rate = 2e-5
number_of_epochs = 1
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
# choosing Adam optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
df = pd.read_csv('train.csv')
df

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5
...,...,...,...,...,...,...,...,...
3906,FFD29828A873,I believe using cellphones in class for educat...,2.5,3.0,3.0,3.5,2.5,2.5
3907,FFD9A83B0849,"Working alone, students do not have to argue w...",4.0,4.0,4.0,4.0,3.5,3.0
3908,FFDC4011AC9C,"""A problem is a chance for you to do your best...",2.5,3.0,3.0,3.0,3.5,3.0
3909,FFE16D704B16,Many people disagree with Albert Schweitzer's ...,4.0,4.5,4.5,4.0,4.5,4.5


In [10]:
def get_label(df):
  df_cohesion = df[["text_id", "full_text", "cohesion"]]
  df_cohesion['rating'] = np.zeros(len(df_cohesion))
  for i in range(1, len(df_cohesion)):
    score = df_cohesion.loc[i, 'cohesion']
    if score >= 3:
        df_cohesion.loc[i, 'rating'] = 1
    else:
        df_cohesion.loc[i, 'rating'] = 0

  return df_cohesion

df_cohesion = get_label(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


https://towardsdatascience.com/how-to-apply-transformers-to-any-length-of-text-a5601410af7f

In [130]:
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def convert_example_to_feature(review): #make dataset to be a square 
  return tokenizer.encode_plus(review,
                add_special_tokens = True, # add [CLS], [SEP]
                max_length = 512, # max length of the text that can go to BERT
                pad_to_max_length = True, # add [PAD] tokens 按照最大长度补齐
                return_attention_mask = True, # add attention mask to not focus on pad tokens
              )
  
def encode_examples():  
  max_len = 0
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  label_list = []

  sentences = df_cohesion["full_text"].values
  labels_raw = df_cohesion["rating"].values

  for k in range(len(sentences)):
      sent = sentences[k]
      tokens = convert_example_to_feature(sent)
      input_ids_list.append(tokens['input_ids'])
      token_type_ids_list.append(tokens['token_type_ids'])
      attention_mask_list.append(tokens['attention_mask'])
      label_list.append([labels_raw[k]])

  dataset = tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, 
                                             token_type_ids_list, label_list)).map(map_example_to_dict)
  return dataset

ds_train_encoded = encode_examples().shuffle(1000).batch(batch_size)
ds_test_encoded = encode_examples().shuffle(100).batch(batch_size)

<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>


In [123]:
bert_history = model.fit(ds_train_encoded, epochs=number_of_epochs, validation_data=ds_test_encoded)

652/652 [==============================] - 98s 151ms/step - loss: 0.3086 - accuracy: 0.8793 - val_loss: 0.1133 - val_accuracy: 0.9739


In [141]:
def getLongestParagraph(df_cohesion):
  lengths = df_cohesion["full_text"].str.len()
  argmax = np.where(lengths == lengths.max())[0]
  return df_cohesion.iloc[argmax]["full_text"].values[0]

def getGrade(numpy_prediction):
  return int(numpy_prediction[0][1]*10) / 2

def predict_grade(txt):
  tokens = tokenizer.encode(txt, truncation=True, padding=True, return_tensors='tf')
  tf_output= model.predict(tokens)[0]
  tf_prediction = tf.nn.softmax(tf_output, axis=1)
  numpy_prediction = tf_prediction.numpy()
  txt_grading = getGrade(numpy_prediction)
  return txt_grading

txt = getLongestParagraph(df_cohesion)
predict_grade(txt)

1/1 [==============================] - 0s 45ms/step


4.5

In [142]:
def computeLoss100(df_cohesion, model):
    loss = 0
    for index, row in df_cohesion.head(100).iterrows():
        txt = row["full_text"]
        predicted_cohesion_score = predict_grade(txt)
        loss = loss + (row["cohesion"] - predicted_cohesion_score) ** 2
    return loss
    
loss = computeLoss100(df_cohesion, model)

1/1 [==============================] - 0s 39ms/step


In [143]:
loss

213.0

In [131]:
def breakintochunks(input_id_chunks, mask_chunks, label): #Not useful by now
    input_id_chunks_ = list(input_id_chunks)
    mask_chunks_ = list(mask_chunks)

    for i in range(len(input_id_chunks_)):
      input_id_chunks_[i] = torch.cat([
          torch.Tensor([101]), input_id_chunks_[i], torch.Tensor([102])
          ])
      mask_chunks_[i] = torch.cat([
          torch.Tensor([1]), mask_chunks_[i], torch.Tensor([1])
          ])
      
      pad_len = max_sequence_length - input_id_chunks_[i].shape[0]
      if pad_len > 0:
        input_id_chunks_[i] = torch.cat([
            input_id_chunks_[i], torch.Tensor([0] * pad_len)
        ])

        mask_chunks_[i] = torch.cat([
            mask_chunks_[i], torch.Tensor([0] * pad_len)
        ])
      #print(len(input_id_chunks_[i]), end = ' ')
    #print()

    input_id = torch.stack(input_id_chunks_)
    attention_mask = torch.stack(mask_chunks_)
    labels_nparray = np.full(len(input_id_chunks_), label)
    labels_tensor = torch.from_numpy(labels_nparray)
    #print(len(input_id), len(attention_mask), len(labels_tensor))
    return input_id, attention_mask, labels_tensor